In [36]:
import numpy as np
from random import *
import tkinter as tk
import tkinter.messagebox

%matplotlib inline 
from matplotlib import pyplot as plt

In [37]:
class Game_2048: 
    """
    The board game 2048 Supreme.
    
    Attributes:
    -----------
    board: the game board
    m: height of the board
    n: width of the board
    goal: the target goal to win the game
    prime: the base number of the game
    game_over: 1 = game over, 0 = game ongoing
    win: whether or not we won
    actions: allowable actions at each timestep
    """
    board = np.zeros(4)
    m = 4
    n = 4
    goal = 2048
    prime = 2
    game_over = 0
    win = 0
    actions = ["left", "up", "right", "down", '0', '1', '2', '3', 
               0, 1, 2, 3, "a", "w", "d", "s"]
    history = []
    
    def __init__(self, m = 4, n = 4, prime = 2, goal = 2048, board_array = None):
        """
        Creates a new instance of the game. Sets up the board, prime number, and goal number. 
        
        By default, it creates a standard 4 x 4 board. 
        
        Inputs
        ------
        m = # rows or the height of the board
        n = # columns or the width of the board
        prime = the base prime number of this game
        goal = the goal number to achieve to win
        board_array = a custom np array to use as a board
        """
        if board_array is None:
            self.board = np.zeros((m, n))
            self.m = m
            self.n = n
        else:
            self.board = board_array
            
        self.goal = goal
        self.prime = prime
        self.game_over = 0
        self.win = 0
        
        for i in range(0, self.prime):
            self.board = self.fill_random_cell(self.board, self.prime)
        
    def fill_random_cell(self, board, prime):
        """
        Fills a random empty cell in the board with the prime value.
        
        Inputs
        ------
        board: board you want to fill a cell with
        prime: the prime number to fill in the board
        
        Outputs:
        --------
        modBoard: the modified board with a cell filled in
        """
        modBoard = np.copy(board)
        i, j = (modBoard == 0).nonzero()
        if i.size != 0:
            rnd = randint(0, i.size - 1)
            modBoard[i[rnd], j[rnd]] = prime
        else:
            self.game_over = 1
        return modBoard
            
    def get_successor_dep(self, board):
        """
        Moves all blocks to the left in the given board 
        
        Proceeds by column, moving each item in the column as far left as possible individually.
        
        If a block encounters another block, then it stops unless they are the same block. If so,
        then it counts if we can add a number of those blocks together equal to prime. If so, then
        they are combined. 
        
        Then it fills a random cell on the board. Then checks if it is game over. If a cell could not
        be filled, then it is game over. 
        
        Inputs:
        -------
        board: the board you want to enact moveLeft on
        
        Outputs:
        --------
        modBoard: the left shifted board
        
        """
        n = self.n
        m = self.m
        modBoard = np.copy(board)
        merged = []
        for i in range(n):
            for j in range(m):
                current = modBoard[j][i]
                if current == 0:
                    continue
                y, x = j, i
                for k in range(1, i+1):
                    block = modBoard[j][i-k]
                    if block == 0:
                        modBoard[j][i-k] = current
                        modBoard[y][x] = 0
                        y = j
                        x = i - k
                        continue
                    elif (block != current) or (j, i-k) in merged:
                        break
                    else:
                        count = 1
                        combine = False
                        zeros = [(y, x)]
                        for c in range(k, i+1): 
                            if modBoard[j][i-c] == current:
                                count += 1
                                if count == self.prime: 
                                    modBoard[j][i-c] = current * self.prime
                                    merged.append((j, i-c))
                                    for index in zeros:
                                        modBoard[index[0]][index[1]] = 0
                                    break
                                zeros.append((j, i-c))
                            else:
                                break
        return modBoard
    
    def get_successor(self, board): 
        """
        Moves all blocks to the left in the given board 
        
        Proceeds by column vector, scanning and shifting the column vector until all items have been placed.
        
        If a block encounters another block, then it stops unless they are the same block. If so,
        then it counts if we can add a number of those blocks together equal to prime. If so, then
        they are combined. 
        
        Then it fills a random cell on the board. Then checks if it is game over. If a cell could not
        be filled, then it is game over. 
        
        Inputs:
        -------
        board: the board you want to enact moveLeft on
        
        Outputs:
        --------
        modBoard: the left shifted board
        """
        
        def merge_vectors(index_1, index_2, board, merge_array):
            """
            Merge two vectors by shifting vector_2 as far left as 
            possible into vector 1.
            
            Returns the new (vector_1, vector_2) after the shift is complete.
            
            Inputs:
            ------
            index_1: the index of the first vector on the board
            index_2: the index of the second vector on the board
            board: the board that you are extracting the vectors from
            merge_array: a 2D array with 1 in entries where we are not allowed to merge
            """
            vec_1 = np.copy(board[:, index_1])
            vec_2 = np.copy(board[:, index_2])
            ill_merge_1 = np.copy(merge_array[:, index_1])
            ill_merge_2 = np.copy(merge_array[:, index_2])
            for i in range(len(vec_1)):
                # Nothing to shift
                if vec_2[i] == 0:
                    continue
                # Space available to shift into
                elif vec_1[i] == 0:
                    vec_1[i] = vec_2[i]
                    vec_2[i] = 0
                # Merge is possible
                # Check not just if equal but if a multiple of a recent prime
                # TODO: WTF is the prime and update the if statement below
                # TODO: RECORD THE LOCATIONS OF THE (ORIGIN MOVED) -> (END MOVED)
                elif (vec_1[i] == vec_2[i]) and (ill_merge_1[i] == 0) and (ill_merge_2[i] == 0):
                    vec_1[i] = vec_2[i] + vec_1[i]
                    vec_2[i] = 0
                    ill_merge_1[i] = 1
                    # the prime is what?
                    # your current merge checking sucks
                    # TODO: FIX HOW YOU CHECK TO NOT MERGE MULTIPLE TIMES
            merge_array[:, index_1] = ill_merge_1
            merge_array[:, index_2] = ill_merge_2
            return vec_1, vec_2, merge_array
        
        rows, columns = board.shape
        mod_board = np.copy(board)
        merge_array = np.zeros((rows, columns))
        for col_index in range(columns):
            #column_vector = board[:, col_index] 
            # array with 0 if a position has been merged, 1 otherwise
            for shift_index in range(col_index):
                index_1 = col_index-shift_index-1
                index_2 = col_index-shift_index
                vec_1, vec_2, new_merge_array = merge_vectors(index_1, index_2, 
                                             mod_board, merge_array)
                """print("vec_1", vec_1)
                print("vec_1 index", col_index-shift_index-1)
                print("vec_2", vec_2)
                print("vec_2 index", col_index-shift_index)"""
                mod_board[:, index_1] = vec_1
                mod_board[:, index_2] = vec_2
                #print("mod_board\n", mod_board)
                merge_array = new_merge_array
        return mod_board
                
    
    def get_action(self, command):
        """
        Takes in a varaible command and returns the corresponding action.
        """
        if command in self.actions:
            movement = self.actions.index(command) % 4
        else:
            movement = None
        return movement
    
    def move(self, command = 0):
        """
        Moves the board in the desired direction. Moving up, right, or down is just
        moving left on a rotated board. 
        
        Inputs:
        -------
        direction: The direction you wish to move, the following are all equivalent.
            "left" = '0' = 0
            "up" = '1' = 1
            "right" = '2' = 2
            "down" = '3' = 3
        """
        if command in self.actions:
            board = np.copy(self.board)
            movement = self.get_action(command)
            if movement == None:
                print('Error, retrieved incorrect command')
                pass  
            board = np.rot90(board, movement)
            board = self.get_successor(board)
            board = np.rot90(board, -movement)
            
            if np.array_equal(board, self.board) == False:
                board = self.fill_random_cell(board, self.prime)
                # adds current state to history before overriding it
                self.history.append(self.board)
                self.board = board
        else:
            print("Error. Please enter one of the following: left, up, right, down. \n \
            Alternatively, type: 0, 1, 2, or 3")
    
    def is_game_over(self):
        """
        Checks if the game is over.
        
        Outputs:
        --------
        1: game is over
        0 : game is ongoing
        """
        if self.game_over == 1 or self.is_win() == 1:
            return 1
        else:
            return 0
        
    def is_win(self):
        """
        Checks if we won the game
        
        Outputs:
        --------
        1: we won
        0: we lost
        """
        for block in np.nditer(self.board):
            if block == self.goal:
                return 1
        return 0
    
    def get_val(self, x, y):
        """
        Returns the value of the square at position (x, y)
        """
        return int(self.board[y][x])
    
    def get_legal_actions(self):
        """
        Returns a list of legal actions
        """
        return self.actions
    
    def get_last_state(self):
        """
        Returns the board state we were in before this one, or None of it does not exist.
        """
        if len(self.history) == 0:
            return None
        else:
            return self.history[-1]
        
    def undo_last_move(self):
        """
        Reverts the board state to the next most recent state in our history, if it exists
        """
        if len(self.history) != 0:
            last_state = self.history.pop()
            self.board = last_state 
            

In [38]:
game = Game_2048(m = 4, n = 4, prime = 2, goal = 32)
x = np.arange(4).reshape((2,2))

In [39]:
print("Starting game now.")
while True:
    #plt.matshow(game.board)
    #plt.show()
    print(game.board)
    if game.is_game_over() == 1:
        if game.is_win() == 0:
            print("Sorry, you lost.")
        else:
            print("You won!")
        break
    action = input()
    if action.lower() == "exit":
        print("Ending game now.")
        break
    game.move(action)

Starting game now.
[[0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]]
left
[[0. 2. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]]
left
[[2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 2. 0.]
 [0. 0. 0. 0.]]
left
[[2. 0. 0. 0.]
 [2. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]]
exit
Ending game now.


In [40]:
class StartGame:
    BG = "#fbf8ef"
    TITLE = "2048 Supreme"
    TITLE_FONT = ("Helvetica", 28)
    SQUARE_FONT = ("Helvetica", 16)
    SQUARE_SIZE = 100
    SQUARE_OUTLINE = "#b9ab9e"
    SQUARE_FILL = {0: "#c9bdb1", 2: "#eee4da", 4: "#ece0c8", 8: "#f2b179", 
                   16: "#f29762", 32: "#f57c5f", 64: "#f65d3b" , 128: "#edce71", 
                   256: "#f9d067", 512: "#e4c02a", 1024: "#e4b714", 
                   2048: "#fac42e"}
    SQUARE_TEXT_COLOR = {0: None, 2: "#796860", 4: "#796860", 8: "white", 16: "white",
                   32: "white", 64: "white", 128: "white", 256: "white", 
                   512: "white", 1024: "white", 2048: "white"}
    
    def __init__(self):
        self.GAME = Game_2048()
        self.numRows = self.GAME.m
        self.numCols = self.GAME.n
        self.boardHeight = self.numRows * self.SQUARE_SIZE
        self.boardWidth = self.numCols * self.SQUARE_SIZE
        
        self.root = tk.Tk()
        self.root.title(self.TITLE)
        self.root.configure(bg=self.BG, width=self.boardWidth * 2, height=self.boardHeight*2)
        
        title=tk.Label(self.root, text=self.TITLE, bg=self.BG, fg="#756e64", font=self.TITLE_FONT)
        title.pack()
        
        
        # Main Board
        boardFrame = tk.Frame(self.root, width=self.boardWidth+20, height=self.boardHeight+20, bg=self.BG, padx=10, pady=10)
        boardFrame.pack()
        
        self.canvas = tk.Canvas(boardFrame, width=self.boardWidth, height=self.boardHeight, bg="#baaea0", borderwidth=0)
        self.canvas.pack()
     
        self.update_board()
        
        # Command Frame and Buttons
        commandFrame=tk.Frame(self.root, width=self.boardWidth, height=100, bg=self.BG, pady=10, padx=10)
        commandFrame.pack()
        
        self.set_command_buttons(commandFrame)
        
        # Return Button
        returnButton = tk.Button(self.root, text="Quit", padx=5, pady=5)
        returnButton.pack(side=tk.BOTTOM)
        returnButton.bind("<Button 1>", self.quit)
        
        self.root.bind_all("<Escape>", self.quit)
        
        self.root.mainloop()
        
    def create_frames(self, root):
        pass
    
    def create_widgets(self):
        pass
    
    def set_command_buttons(self, frame):
        upButton=tk.Button(frame, text="Up", padx=5, pady=5)
        leftButton=tk.Button(frame, text="Left", padx=5, pady=5)
        rightButton=tk.Button(frame, text="Right", padx=5, pady=5)
        downButton=tk.Button(frame, text="Down", padx=5, pady=5)
        
        upButton.bind("<Button 1>", self.move)
        leftButton.bind("<Button 1>", self.move)
        rightButton.bind("<Button 1>", self.move)
        downButton.bind("<Button 1>", self.move)
        
        upButton.pack(side=tk.TOP)
        leftButton.pack(side=tk.LEFT)
        rightButton.pack(side=tk.RIGHT)
        downButton.pack(side=tk.BOTTOM)
        
        self.root.bind_all("<w>", self.move)
        self.root.bind_all("<a>", self.move)
        self.root.bind_all("<s>", self.move)
        self.root.bind_all("<d>", self.move)
        
        self.root.bind_all("<Up>", self.move)
        self.root.bind_all("<Left>", self.move)
        self.root.bind_all("<Down>", self.move)
        self.root.bind_all("<Right>", self.move)
        
    def move(self, event):
        if event.char == "??":
            command = str(event.widget.cget("text")).lower().strip()
        else:
            command = str(event.keysym).lower().strip()
        if command in self.GAME.get_legal_actions():
            self.GAME.move(command)
            self.update_board()
        
    def quit(self, event):
        self.root.destroy()
    
    def update_board(self):
        for col in range(self.numCols):
            for row in range(self.numRows):
                num = self.GAME.get_val(x=col, y=row)
                if num in self.SQUARE_FILL:
                    num_bg = self.SQUARE_FILL[num]
                    num_text_color = self.SQUARE_TEXT_COLOR[num]
                else:
                    num_bg = self.SQUARE_FILL[2048]
                    num_text_color = self.SQUARE_TEXT_COLOR[2048]
                    
                self.canvas.create_rectangle(self.SQUARE_SIZE*col, self.SQUARE_SIZE*row, 
                                             self.SQUARE_SIZE*(col+1), self.SQUARE_SIZE*(row+1), 
                                             outline=self.SQUARE_OUTLINE, fill=num_bg)
                if num != 0:
                    self.canvas.create_text(self.SQUARE_SIZE*(col+0.5), self.SQUARE_SIZE*(row+0.5), 
                                            text=str(num), font=self.SQUARE_FONT, fill=num_text_color)
        

In [41]:
game = StartGame()

In [ ]:
array = np.array([1, 2, 3, 4])
array_t = np.transpose(array)
print(array)
print(array_t)
print(array.shape)
print(array_t.shape)